## Script to scrape salary, level, years of experience and years at company data from levels.fyi

In [1]:
#Import library
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
def extract_content(soup,search_term):
    '''
    INPUT
    soup - output of beautiful soup html parser
    search_term - list (size 3) of search terms
    
    OUTPUT
    content - python list of scrapped data based on input search term
    
    This function cleans df using the following steps to produce clean df:
        1. Narrow soup search to level of table (<td> tag)
        2. Loop over <tr> tag in the table using find_all
        3. extract contents using <span> tag
        4. append to list 
        
    e.g. usage
    for the website : https://www.levels.fyi/comp.html?track=Product%20Manager
    
    level_search_term = ["td","class","text-left otherWidth"]    
    exp_search_term = ["td","class","text-left otherWidth hidden-xs hidden-sm"]    
    salary_search_term = ["td","class","text-right otherWidth"]
    
    '''
    
    l1_srch_soup = soup.find("table", {"id": "compTable"})
    
    l2_srch_soup = l1_srch_soup.find_all(search_term[0], 
                                         
                                         {search_term[1]: search_term[2]})
           
    content = [x.find('span').contents[0].__str__() for x in l2_srch_soup]
    
    return content

def extract_salary_breakdown(soup,search_term):
    '''
    INPUT
    soup - output of beautiful soup html parser
    search_term - list (size 3) of search terms
    
    OUTPUT
    content - python list of scrapped salary breakdown (Base|Stock|Bonus)
    
    This function cleans df using the following steps to produce clean df:
        1. Narrow soup search to level of table (<td> tag)
        2. Loop over <tr> tag in the table using find_all
        3. extract contents using <span> tag
        4. append to list 
        
    e.g. usage
    for the website : https://www.levels.fyi/comp.html?track=Product%20Manager
     
    salary_search_term = ["td","class","text-right otherWidth"]
    
    '''
    
    l1_srch_soup = soup.find("table", {"id": "compTable"})
    
    l2_srch_soup = l1_srch_soup.find_all(search_term[0], {search_term[1]:
                                                          
                                                          search_term[2]})
    
    content = [x.find('span',{'class':"dateDetails" }).contents[0].__str__() 
               
               for x in l2_srch_soup if x.find('span',{'class':"dateDetails" }) is not None]
    
    return content

## Step 1 

In [3]:
### Start driver. Chromium driver exec is located in path modify executable_path to where driver is located
driver = webdriver.Chrome(
    executable_path='D:/Tools/MyTools/Drivers/chromedriver.exe'
    ) 

## Step 2 

Navigate to https://www.levels.fyi/comp.html?track=Product%20Manager  in the newly opened browser window

## Step 3 

Click on Show upto 100 lines per table

![levelfyi](levelsfyi2.png)

## Step 4 

In [4]:
### Initialize lists
level = []
exp =[]
salary = []
salary_breakdown = []

## Step 5  

In [5]:
### Scrape data from the current table
content = driver.page_source
soup = BeautifulSoup(content,'html.parser')
   
level_search_term = ["td","class","text-left otherWidth"]    
exp_search_term = ["td","class","text-left otherWidth hidden-xs hidden-sm"]    
salary_search_term = ["td","class","text-right otherWidth"]

level = level + extract_content(soup,level_search_term)
exp = exp + extract_content(soup,exp_search_term)
salary = salary + extract_content(soup,salary_search_term)
salary_breakdown = salary_breakdown + extract_salary_breakdown(soup,
                                                               salary_search_term)
##### 

## Step 6  

Go to next page. Repeat Step 5 (The code in step 5 appends the new table to the existing list)

## Step 7

Repeat Step 6 till all pages are scraped

## Step 8 

In [6]:
###Clean data and export as csv using Dataframe
def export_csv(level,exp,salary,salary_breakdown,filename):
    '''
    INPUT
    level - list of levels scraped from levelfyi site
    exp - list of experience scraped from levelfyi site
    salary - list of salary scraped from levelsfyi site
    salary_breakdown - list of salary breakdown  from levelsfyi site
    filename - filename of csv
    
    OUTPUT
    df - combined dataframe
    
    This function
    1. combines all lists into a single datframe
    2. Clean experience and compensation breakdown columns 
    
    '''
    
    df = pd.DataFrame(list(zip(level, exp,salary,salary_breakdown)), 
               columns =['Level','Experience','Total Compensation',
                         'Compensation Breakdown (Base|Stock|Bonus)'])

    df[['Years at Company','Years of Experience']] = df['Experience'].str.split('/',expand=True)
    df[['Base','Stock','Bonus']] = df['Compensation Breakdown (Base|Stock|Bonus)'].str.split('|',expand=True)
    
    df = df.drop(columns = ['Experience','Compensation Breakdown (Base|Stock|Bonus)'])
    
    df.to_csv(filename+'.csv')
    
    return df
                            
#export data
export_csv(level,exp,salary,salary_breakdown,'Salary_Data')  

,Level,Total Compensation,Years at Company,Years of Experience,Base,Stock,Bonus
0,L60,"$176,000",0,2,119k,37k,20k
1,L7,"$400,000",10,15,160k,240k,0k
2,7,"$430,000",10,15,150k,280k,0k
3,L4,"$134,000",7,7,134k,0k,5k
4,Director,"$313,000",1,8,210k,60k,42k
...,...,...,...,...,...,...,...
95,64,"$220,000",15,15,170k,25k,25k
96,L6,"$276,000",3,8,136k,140k,0k
97,L7,"$350,000",2,12,260k,60k,28k
98,L6,"$197,000",2,5,143k,31k,23k


## Step 9

In [8]:
## Quit the driver
driver.quit()